## Alireza Farajtabrizi (97102185) Lab4


In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

## Part 1

### define a function to make labels and split train and test set

In [2]:
from os import listdir
from skimage import io
from sklearn.model_selection import train_test_split
def load_and_split(folder_dir, train_size, label = 0):
    X = []
    for file in listdir(folder_dir):
        img=io.imread(folder_dir+'/'+file,as_gray=True)
        X.append(img.reshape(img.shape[0]*img.shape[1]))          
    
    y = np.full(len(X), label)
    X = np.array(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=train_size ,random_state=np.random.randint(100))
    return X_train, X_test, y_train, y_test

### using the to function to  build train and test set

In [3]:

x_n_train, x_n_test, y_n_train, y_n_test = load_and_split('Neutral', 120, 0)
x_h_train, x_h_test, y_h_train, y_h_test = load_and_split('Happy', 60, 1)
x_s_train, x_s_test, y_s_train, y_s_test = load_and_split('Surprise', 60, 2)

x_train = np.concatenate((x_n_train, x_h_train, x_s_train), axis = 0)
x_test  = np.concatenate((x_n_test, x_h_test, x_s_test), axis = 0)
y_train = np.concatenate((y_n_train, y_h_train, y_s_train))
y_test = np.concatenate((y_n_test, y_h_test, y_s_test))

### PCA and Scaler

In [4]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

n=50

pca = PCA(n_components=n)
pca.fit(x_train)

x_train = pca.transform(x_train)
x_test  = pca.transform(x_test)

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test  = scaler.transform(x_test)

### Linear SVM and evaluation of it

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support


c = 0.01

clf = SVC(kernel='linear', C=c)
clf.fit(x_train, y_train)

y_train_P= clf.predict(x_train)
y_test_P= clf.predict(x_test)

print("Confusion Matrix for train : \n" , confusion_matrix(y_train, y_train_P))
print("Confusion Matrix for test : \n" , confusion_matrix(y_test, y_test_P))


persicion_train = precision_recall_fscore_support(y_train, y_train_P)[0]
persicion_test  = precision_recall_fscore_support(y_test, y_test_P)[0]
recall_train = precision_recall_fscore_support(y_train, y_train_P)[1]
recall_test  = precision_recall_fscore_support(y_test, y_test_P)[1]

print("Persicion of train for each class : \t", persicion_train , "\tand on average : \t" , np.mean(persicion_train))
print("Persicion of test for each class : \t", persicion_test ,"\tand on average : \t" , np.mean(persicion_test))
print("Recall of train for each class : \t", recall_train,"\tand on average : \t" , np.mean(recall_train))
print("Recall of test for each class : \t", recall_test,"\tand on average : \t" , np.mean(recall_test))


Confusion Matrix for train : 
 [[120   0   0]
 [ 11  49   0]
 [ 17   0  43]]
Confusion Matrix for test : 
 [[70  0  0]
 [ 7 17  1]
 [ 7  0 21]]
Persicion of train for each class : 	 [0.81081081 1.         1.        ] 	and on average : 	 0.936936936936937
Persicion of test for each class : 	 [0.83333333 1.         0.95454545] 	and on average : 	 0.9292929292929294
Recall of train for each class : 	 [1.         0.81666667 0.71666667] 	and on average : 	 0.8444444444444444
Recall of test for each class : 	 [1.   0.68 0.75] 	and on average : 	 0.81


### define a cross-validation function to get best C

In [6]:
from sklearn.metrics import accuracy_score
def find_C(X, y):
    
    C_range = np.logspace(-10, 10, 21)
    acc = []
    for c in C_range:
        x_train , x_test, y_train, y_test =  train_test_split(X, y, train_size=0.80, random_state=np.random.randint(100))
        clf = SVC(kernel='linear', C=c)
        clf.fit(x_train, y_train)
        acc.append(accuracy_score(y_test, clf.predict(x_test)))
    return C_range[np.argmax(acc)]

### running the validation function several times to avoid the randomize effect

In [11]:
C=[]
for i in range(100):
    C.append(find_C(x_train, y_train))
best_c = max(set(C), key = C.count)
print("the best C is :" , best_c)

the best C is : 1.0


if the above result is not 0.1 or 1 it means the the data set had a bad shuffle and you need to run again

### Linear SVM with best C and evaluation of it

In [8]:
c = best_c

clf = SVC(kernel='linear', C=c)
clf.fit(x_train, y_train)

y_train_P= clf.predict(x_train)
y_test_P= clf.predict(x_test)

print("Confusion Matrix for train : \n" , confusion_matrix(y_train, y_train_P))
print("Confusion Matrix for test : \n" , confusion_matrix(y_test, y_test_P))


persicion_train = precision_recall_fscore_support(y_train, y_train_P)[0]
recall_train = precision_recall_fscore_support(y_train, y_train_P)[1]
persicion_test = precision_recall_fscore_support(y_test, y_test_P)[0]
recall_test = precision_recall_fscore_support(y_test, y_test_P)[1]

print("Persicion of train for each class : \t", persicion_train , "\tand on average : \t" , np.mean(persicion_train))
print("Persicion of test for each class : \t", persicion_test ,"\tand on average : \t" , np.mean(persicion_test))
print("Recall of train for each class : \t", recall_train,"\tand on average : \t" , np.mean(recall_train))
print("Recall of test for each class : \t", recall_test,"\tand on average : \t" , np.mean(recall_test))

Confusion Matrix for train : 
 [[120   0   0]
 [  0  60   0]
 [  0   0  60]]
Confusion Matrix for test : 
 [[65  3  2]
 [ 2 23  0]
 [ 4  0 24]]
Persicion of train for each class : 	 [1. 1. 1.] 	and on average : 	 1.0
Persicion of test for each class : 	 [0.91549296 0.88461538 0.92307692] 	and on average : 	 0.9077284218129288
Recall of train for each class : 	 [1. 1. 1.] 	and on average : 	 1.0
Recall of test for each class : 	 [0.92857143 0.92       0.85714286] 	and on average : 	 0.901904761904762


as you can see the Persicion has decreased by 0.02 and Recall is increased by 0.08 and the number of miss predictions has decreased by 4
since the best c is not that different from default or some times its the same you dont see that much approvement if you put the c bellow 10^(-2) you will see how bad the result will become

## Part 2

### Grid Search

In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

C_range = np.logspace(-10, 10, 21)
gamma_range = np.logspace(-10, 10, 21)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(x_train, y_train)

print(
    "The best parameters are %s with a score of %0.2f"
    % (grid.best_params_, grid.best_score_)
)

The best parameters are {'C': 1000.0, 'gamma': 0.0001} with a score of 0.93


### RBF SVM with best parameters and evaluation of it

In [10]:
C, gamma = grid.best_params_['C'], grid.best_params_['gamma']
clf = SVC(C=C, gamma=gamma)
clf.fit(x_train, y_train)
y_train_P= clf.predict(x_train)
y_test_P= clf.predict(x_test)

print("Confusion Matrix for train : \n" , confusion_matrix(y_train, y_train_P))
print("Confusion Matrix for test : \n" , confusion_matrix(y_test, y_test_P))


persicion_train = precision_recall_fscore_support(y_train, y_train_P)[0]
recall_train = precision_recall_fscore_support(y_train, y_train_P)[1]
persicion_test = precision_recall_fscore_support(y_test, y_test_P)[0]
recall_test = precision_recall_fscore_support(y_test, y_test_P)[1]

print("Persicion of train for each class : \t", persicion_train , "\tand on average : \t" , np.mean(persicion_train))
print("Persicion of test for each class : \t", persicion_test ,"\tand on average : \t" , np.mean(persicion_test))
print("Recall of train for each class : \t", recall_train,"\tand on average : \t" , np.mean(recall_train))
print("Recall of test for each class : \t", recall_test,"\tand on average : \t" , np.mean(recall_test))

Confusion Matrix for train : 
 [[120   0   0]
 [  0  60   0]
 [  1   0  59]]
Confusion Matrix for test : 
 [[67  1  2]
 [ 2 23  0]
 [ 4  0 24]]
Persicion of train for each class : 	 [0.99173554 1.         1.        ] 	and on average : 	 0.997245179063361
Persicion of test for each class : 	 [0.91780822 0.95833333 0.92307692] 	and on average : 	 0.9330728251961129
Recall of train for each class : 	 [1.         1.         0.98333333] 	and on average : 	 0.9944444444444445
Recall of test for each class : 	 [0.95714286 0.92       0.85714286] 	and on average : 	 0.9114285714285715


the result of RBF SVM comparing to Linear SVM is better (not much).the Persicion is increased by 0.03 and the recall is increased by 0.02 and number of miss predictions is decreased by 2 